In [0]:
dbutils.library.installPyPI("mlflow")
dbutils.library.restartPython()

PyPI package mlflow has been installed already. The previously-installed package is `mlflow`. To resolve this issue detach and re-attach the notebook to create a new environment or rename the package.

In [0]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import mlflow
from sklearn.preprocessing import StandardScaler
from IPython.display import display, HTML
from sklearn.ensemble import IsolationForest


In [0]:
#Iforest model from Keras
class IForest:

    def __init__(self, scaled_user_ts, contamination=0.01, random_state=0, verbose=0, max_features=1.0):
    
        self.model =  IsolationForest(n_estimators=100, contamination=contamination, random_state=random_state, verbose=verbose, max_features=max_features)

        #scaler = StandardScaler()
        self.model.fit(scaled_user_ts)
    
    def predict(self, ts):
        return self.model.predict(ts)#, abs(self.model.decision_function(ts)) * 100
      

In [0]:
#Convert spark dataframe to pandas dataframe
data = spark.table('better_agg').toPandas()

In [0]:
# user user_id as index and delete it from columns. Delete timestamp as well
data.index = data["user_id"]
del data["user_id"]
del data["timestamp"]


In [0]:
#Model parameter assignment
contamination=0.01
random_state=0
verbose=0
max_features = 1.0

In [0]:

#with mlflow.start_run(run_name="YOUR_RUN_NAME") as run:
with mlflow.start_run():
  mlflow.log_param("contamination",contamination)
  mlflow.log_param("max_features",max_features)
  
  #Training Data
  iforest = IForest(data,contamination,random_state,verbose,max_features)



In [0]:
  #predict using the same data as we are looking for anomalies in this data
  pred = iforest.predict(data)


In [0]:
  #assign score and prediction back to the dataframe
  #score = pd.DataFrame(pred[1])
  pred = pd.DataFrame(pred)
  data["anomaly"] = pred.values
 #"data["score"] = score.values


In [0]:
  # Convert into Spark DataFrame and convert it into table
  spark_df = spark.createDataFrame(data)
  spark_df.write.mode("overwrite").saveAsTable("better_ip_results")

In [0]:
  #mlflow.log_metric("#anomalies", len(data[data['anomaly'] == -1]['anomaly']), step=None)

  mlflow.sklearn.log_model(
          sk_model=iforest,
          artifact_path="sklearn-model",
          registered_model_name="i_Forest"
      )


Successfully registered model 'i_Forest'.
Created version '1' of model 'i_Forest'.

In [0]:
#mlflow.spark.save_model("/users/pedro.neves/models/isolationForest")

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-1977472393598305> in <module> 
 ----> 1 mlflow . spark . save_model ( "/users/pedro.neves/models/isolationForest" ) 

 TypeError : save_model() missing 1 required positional argument: 'path'

In [0]:
mlflow.end_run()
